In [3]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https:/www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Run experiments with Pipelines


## Overview

Vertex Pipelines will allow you to experiment and track Training Pipeline runs and its associated parameters.
You can  compare runs of these Pipelines to each others in order to figure out which is the best configuration generates the model you will register in the Vertex AI Model Registry.


### Objective

In this notebook, you learn how to use `Vertex AI Experiments` and `Vertex AI Pipelines` to log a pipeline job and compare different pipeline jobs.

The steps covered include:

* Formalize a training component
* Build a training pipeline
* Run several Pipeline jobs and log their results
* Compare different Pipeline jobs

### Dataset

In the following example, you train a simple distributed neural network (DNN) model to predict automobile's miles per gallon (MPG) based on automobile information in the [auto-mpg dataset](https://www.kaggle.com/devanshbesain/exploration-and-analysis-auto-mpg).


## Before you begin

#### Set variables



In [21]:
GCP_PROJECTS = !gcloud config get-value project
PROJECT_ID = GCP_PROJECTS[0]
BUCKET_URI = f"gs://{PROJECT_ID}-mlops" 
REGION = "us-central1"

# Experiments
EXPERIMENT_NAME = "test-experiment"

# Pipeline
COMPILED_PIPELINE_FILE = "pipeline.json"
PIPELINE_ROOT = f"{BUCKET_URI}/pipelines"

### Import libraries and define constants

In [22]:
import logging
# General
import os
import time

logger = logging.getLogger("logger")
logging.basicConfig(level=logging.INFO)
from kfp.v2.dsl import Model, importer

import kfp.v2.compiler as compiler
# Pipeline Experiments
import kfp.v2.dsl as dsl
# Vertex AI
from google.cloud import aiplatform
from google.cloud.aiplatform_v1.types.pipeline_state import PipelineState
from kfp.v2.dsl import Metrics, Model, Output, component, Input, Dataset, Condition, Artifact, HTML
from google_cloud_pipeline_components.types import artifact_types
from google_cloud_pipeline_components.v1.model import ModelUploadOp
from google_cloud_pipeline_components.v1.endpoint import EndpointCreateOp, ModelDeployOp
from kfp.v2.components import importer_node

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [23]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

## Formalize your training as pipeline component


Before you start running your pipeline experiments, you have to formalize your training as pipeline component.

To do that, you build the pipeline by using the `kfp.v2.dsl.component` decorator to convert the typical steps of a training pipeline into pipeline components. 

In [24]:
@component(
    packages_to_install=["pandas", "tensorflow"]
)
def extract_splits_op(
    data_url: str,
    training_split: Output[Dataset],
    test_split: Output[Dataset],
    split_frac: float =0.8, 
    random_state: int = 0
):  
    from tensorflow.python.keras.utils import data_utils
    import pandas as pd
    dataset_path = data_utils.get_file("auto-mpg.data", data_url)
    column_names = [
        "MPG",
        "Cylinders",
        "Displacement",
        "Horsepower",
        "Weight",
        "Acceleration",
        "Model Year",
        "Origin",
    ]
    raw_dataset = pd.read_csv(
        dataset_path,
        names=column_names,
        na_values="?",
        comment="\t",
        sep=" ",
        skipinitialspace=True,
    )
    dataset = raw_dataset.dropna()
    dataset["Origin"] = dataset["Origin"].map(
        lambda x: {1: "USA", 2: "Europe", 3: "Japan"}.get(x)
    )
    dataset = pd.get_dummies(dataset, prefix="", prefix_sep="")
    train_dataset = dataset.sample(frac=split_frac, random_state=random_state)
    test_dataset = dataset.drop(train_dataset.index)
    train_dataset.to_csv(training_split.path, index=False)
    test_dataset.to_csv(test_split.path, index=False)


In [25]:
@component(
    packages_to_install=[
        "pandas",
        "gcsfs",
        "tensorflow==2.8.0"
    ]
)
def custom_trainer_op(
    training_split: Input[Dataset],
    num_units: int,
    epochs: int,
    dropout_rate:float,
    model: Output[Model],
) -> str:
    import pandas as pd
    from tensorflow.python.keras import Sequential, layers
    from tensorflow.python.keras.utils import data_utils    
    
    def normalize_train_dataset(train_dataset):
        train_stats = train_dataset.describe()
        train_stats = train_stats.transpose()
        def norm(x):
            return (x - train_stats["mean"]) / train_stats["std"]
        normed_train_data = norm(train_dataset)

        return normed_train_data
    
    
    def train(
        train_data,
        train_labels,
        num_units=64,
        activation="relu",
        dropout_rate=0.0,
        validation_split=0.2,
        epochs=1000,
    ):

        model = Sequential(
            [
                layers.Dense(
                    num_units,
                    activation=activation,
                    input_shape=[len(train_dataset.keys())],
                ),
                layers.Dropout(rate=dropout_rate),
                layers.Dense(num_units, activation=activation),
                layers.Dense(1),
            ]
        )
        model.compile(loss="mse", optimizer="adam", metrics=["mae", "mse"])
        print(model.summary())

        history = model.fit(
            train_data, train_labels, epochs=epochs, validation_split=validation_split
        )

        return model, history
    
    # Preprocessing ----------------------------------------------

    train_dataset =  pd.read_csv(training_split.path)
    train_labels = train_dataset.pop("MPG")

    normed_train_data = normalize_train_dataset(train_dataset)
    
    # Train ----------------------------------------------
    model_obj, history = train(
        normed_train_data,
        train_labels,
        num_units=num_units,
        activation="relu",
        epochs=epochs,
        dropout_rate=dropout_rate
    )
    model_obj.save(model.uri)
    return model.uri
    

In [26]:
@component(
    packages_to_install=[
        "pandas",
        "gcsfs",
        "tensorflow==2.8.0"
    ]
)
def evaluate_op(
    training_split: Input[Dataset],
    test_split: Input[Dataset],
    model: Input[Model],
    metrics_metadata: Output[Metrics],
)-> float:
    import pandas as pd
    from tensorflow.python.keras import Sequential, layers
    from tensorflow.python.keras.utils import data_utils    
    from tensorflow import keras
    def normalize_test_dataset(train_dataset, test_dataset):
        train_stats = train_dataset.describe()
        train_stats = train_stats.transpose()
        def norm(x):
            return (x - train_stats["mean"]) / train_stats["std"]
        normed_test_data = norm(test_dataset)

        return normed_test_data

    # Preprocess data ----------------------------------------------

    train_dataset =  pd.read_csv(training_split.path)
    train_labels = train_dataset.pop("MPG")
    test_dataset =  pd.read_csv(test_split.path)
    test_labels = test_dataset.pop("MPG")
    normed_test_data = normalize_test_dataset(train_dataset, test_dataset)
    
    # Load model from disk ----------------------------------------------

    model_object = keras.models.load_model(model.path)

    # Evaluate ----------------------------------------------
    loss, mae, mse = model_object.evaluate(normed_test_data, test_labels, verbose=2)

    metrics_metadata.log_metric("test_loss", loss)
    metrics_metadata.log_metric("test_mae", mae)
    metrics_metadata.log_metric("test_mse", mse)
    return mae
    


## Build a pipeline

We create here the pipeline which will assemble all the several components defined above.
The pipeline will:
1. Extract the data from an URL
2. Train a Tensorflow Model
3. Evaluate the model on a test split

In [27]:
@dsl.pipeline(name="custom-training-pipeline")
def pipeline(
    epochs: int = 2,
    dropout_rate: float = 0.1,
    num_units: int = 16
):
    extract_splits_task = extract_splits_op(data_url="http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data")   
    train_task = custom_trainer_op(
            training_split=extract_splits_task.outputs['training_split'],
            epochs=epochs,
            dropout_rate=dropout_rate,
            num_units=num_units
#    We can specify different CPU, Memory and GPU requirements for every task of our pipeline.     
    ).set_cpu_limit('4').set_memory_limit('16G')
    
    mae_model = evaluate_op(
            training_split=extract_splits_task.outputs['training_split'],
            test_split=extract_splits_task.outputs['test_split'],
            model=train_task.outputs['model'],
    ).outputs["output"]

### Task 1: Submit your first training pipeline
Submit your first training pipeline using all the default parameters and explore the result in the Console. To do that you can click the Pipeline Job link generated by the Vertex AI SDK.

In [28]:
compiler.Compiler().compile(pipeline_func=pipeline, package_path=COMPILED_PIPELINE_FILE)

/home/jupyter/.local/lib/python3.7/site-packages/kfp/v2/compiler/compiler.py:1293: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  category=FutureWarning,


In [29]:
job = aiplatform.PipelineJob(
        display_name=f"pipeline-run",
        template_path=COMPILED_PIPELINE_FILE,
        pipeline_root=PIPELINE_ROOT,
        parameter_values={
# You can add parameters to the pipeline here. See an example below

        },
    )
job.submit()






'''
Solution of pipeline triggered with parameters:

job = aiplatform.PipelineJob(
        display_name=f"pipeline-run",
        template_path=COMPILED_PIPELINE_FILE,
        pipeline_root=PIPELINE_ROOT,
        parameter_values={
            "num_units": 32, 
            "epochs": 10, 
            "dropout_rate": 0.1
        },
    )
job.submit()
'''

Creating PipelineJob


INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob


PipelineJob created. Resource name: projects/370018035372/locations/us-central1/pipelineJobs/custom-training-pipeline-20230123183930


INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/370018035372/locations/us-central1/pipelineJobs/custom-training-pipeline-20230123183930


To use this PipelineJob in another session:


...

pipeline_job = aiplatform.PipelineJob.get('projects/370018035372/locations/us-central1/pipelineJobs/custom-training-pipeline-20230123183930')


INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/370018035372/locations/us-central1/pipelineJobs/custom-training-pipeline-20230123183930')


View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/custom-training-pipeline-20230123183930?project=370018035372


INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/custom-training-pipeline-20230123183930?project=370018035372


'\nSolution of pipeline triggered with parameters:\n\njob = aiplatform.PipelineJob(\n        display_name=f"pipeline-run",\n        template_path=COMPILED_PIPELINE_FILE,\n        pipeline_root=PIPELINE_ROOT,\n        parameter_values={\n            "num_units": 32, \n            "epochs": 10, \n            "dropout_rate": 0.1\n        },\n    )\njob.submit()\n'

### Task 2: Experiment with pipelines

Now that your pipeline is ready to train ML models, you can define its training configuration based on parameters. Below you have an example and how you can submit several pipeline runs with different parameters. 
Once the pipelines are triggered you can explore the results by clicking each individual pipeline or by looking at the [generated experiment page](https://console.cloud.google.com/vertex-ai/locations/us-central1/experiments/test-experiment) to have an overall view of the experiment itself, grouping all the different pipelines.

In [30]:
parameters_list = [
    {"num_units": 16, "epochs": 3, "dropout_rate": 0.1},
    {"num_units": 16, "epochs": 10, "dropout_rate": 0.1},
    {"num_units": 16, "epochs": 10, "dropout_rate": 0.2},
    {"num_units": 32, "epochs": 10, "dropout_rate": 0.1},
    {"num_units": 32, "epochs": 10, "dropout_rate": 0.2},
]

In [31]:
for i, parameters in enumerate(parameters_list):

    job = aiplatform.PipelineJob(
        display_name=f"{EXPERIMENT_NAME}-pipeline-run-{i}",
        template_path=COMPILED_PIPELINE_FILE,
        pipeline_root=PIPELINE_ROOT,
        parameter_values=parameters,
    )
#     We set the experiment name before submitting the pipeline, so that this pipeline can be associated to the experiment.
    job.submit(experiment=EXPERIMENT_NAME)
    

Creating PipelineJob


INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob


PipelineJob created. Resource name: projects/370018035372/locations/us-central1/pipelineJobs/custom-training-pipeline-20230123183936


INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/370018035372/locations/us-central1/pipelineJobs/custom-training-pipeline-20230123183936


To use this PipelineJob in another session:


...

INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/370018035372/locations/us-central1/pipelineJobs/custom-training-pipeline-20230123183948')


View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/custom-training-pipeline-20230123183948?project=370018035372


INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/custom-training-pipeline-20230123183948?project=370018035372


Associating projects/370018035372/locations/us-central1/pipelineJobs/custom-training-pipeline-20230123183948 to Experiment: test-experiment


INFO:google.cloud.aiplatform.metadata.experiment_resources:Associating projects/370018035372/locations/us-central1/pipelineJobs/custom-training-pipeline-20230123183948 to Experiment: test-experiment


## Extending the pipeline

We now extend the pipeline to add new components to it.

- If the pipeline parameter "profile_data" is not "False" the pipeline will perform a profile of the training data
- If the MAE of the model is below 100 and the pipeline parameter "push_model" is not "False" the pipeline will perform a deployment of the model to an Endpoint to allow low latency serving.

#### Data validation and profiling components

As part of an ML pipeline it's important to monitor the data you use to train and serve your model.
In this example we use TFX Tensorflow Data Validation, a comprehensive python package which allows you to perform:
1. Data Profiling: automatically produce reports on your training data to understand the distributions of your features.
2. Data Validation: set expectations on your training data to make sure you only train on good quality data.
3. Training Serving Skew detection: compare training and serving data to detect skew and potentially retrain the model.

We create here 2 components to generate a profile of the data and to visualise a comparison between the train and the test split.

In [32]:
@component(
    packages_to_install=[
        "tensorflow-data-validation"
    ]
)
def generate_statistics_op(
    dataset: Input[Dataset],
    statistics: Output[Artifact],
    statistics_view: Output[HTML]
):
    import tensorflow_data_validation as tfdv
    from tensorflow_data_validation.utils.display_util import get_statistics_html

    dataset_statistics =  tfdv.generate_statistics_from_csv(
        data_location=dataset.uri, output_path=statistics.uri
    )

    html_content = get_statistics_html(lhs_statistics=dataset_statistics)
    statistics_view.path = f"{statistics_view.path}.html"
    with open(statistics_view.path, "w") as f:
        f.write(html_content)

@component(
    packages_to_install=[
        "tensorflow-data-validation"
    ]
)
def generate_statistics_view_comparison_op(
    lhs_statistics: Input[Artifact],
    rhs_statistics: Input[Artifact],
    statistics_view: Output[HTML],
    lhs_name: str = "lhs_statistics",
    rhs_name: str = "rhs_statistics"
):
    import tensorflow_data_validation as tfdv
    from tensorflow_data_validation.utils.display_util import get_statistics_html

    lhs_statistics = tfdv.load_statistics(input_path=lhs_statistics.uri)
    rhs_statistics = tfdv.load_statistics(input_path=rhs_statistics.uri)
    html_content = get_statistics_html(
        lhs_statistics=lhs_statistics,
        rhs_statistics=rhs_statistics,
        lhs_name=lhs_name,
        rhs_name=rhs_name,
    )

    with open(statistics_view.path, "w") as f:
        f.write(html_content)

In [33]:
@dsl.pipeline(name="custom-training-pipeline")
def pipeline(
    project: str = PROJECT_ID,
    epochs: int = 2,
    dropout_rate: float = 0.1,
    num_units: int = 16,
    region: str ="us-central1",
    profile_data: str = "False",
    push_model: str = "False"
):
    extract_splits_task = extract_splits_op(data_url="http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data")   
    train_task = custom_trainer_op(
            training_split=extract_splits_task.outputs['training_split'],
            epochs=epochs,
            dropout_rate=dropout_rate,
            num_units=num_units
#    We can specify different CPU, Memory and GPU requirements for every task of our pipeline.     
    ).set_cpu_limit('4').set_memory_limit('16G')
    
    mae_model = evaluate_op(
            training_split=extract_splits_task.outputs['training_split'],
            test_split=extract_splits_task.outputs['test_split'],
            model=train_task.outputs['model'],
    ).outputs["output"]
    
    with Condition(
        profile_data != "False",
        name="Profile data",
    ):
        training_statistics_op = generate_statistics_op(dataset=extract_splits_task.outputs['training_split'])
        test_statistics_op = generate_statistics_op(dataset=extract_splits_task.outputs['test_split'])
        generate_statistics_view_comparison_op(
            lhs_statistics=training_statistics_op.outputs['statistics'], 
            rhs_statistics=test_statistics_op.outputs['statistics'],
            lhs_name="train_statistics",
            rhs_name="test_statistics"
        )
   
    with Condition(
        mae_model < 100,
        name="MAE is below threshold",
    ):
        with Condition(
            push_model != "False",
            name="push model is below threshold",
        ):
            managed_model = importer_node.importer(
                artifact_uri=train_task.outputs['output'],
                artifact_class=artifact_types.UnmanagedContainerModel,
                metadata={
                    "containerSpec": {
                        "imageUri": "us-docker.pkg.dev/cloud-aiplatform/prediction/tf2-cpu.2-9:latest"
                    }
                },
            ).outputs["artifact"]

            model_upload_op = ModelUploadOp(
                project=PROJECT_ID,
                display_name="mlops_model",
                unmanaged_container_model=managed_model,
            )


            endpoint_op = EndpointCreateOp(
                project=project, location=region, display_name="mlops_model_endpoint"
            )
            _ = ModelDeployOp(
                        model=model_upload_op.outputs["model"],
                        endpoint=endpoint_op.outputs["endpoint"],
                        dedicated_resources_machine_type="n1-standard-4",
                        dedicated_resources_min_replica_count=1,
                        dedicated_resources_max_replica_count=1,
                    )

In [34]:
compiler.Compiler().compile(pipeline_func=pipeline, package_path=COMPILED_PIPELINE_FILE)

/home/jupyter/.local/lib/python3.7/site-packages/kfp/v2/compiler/compiler.py:1293: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  category=FutureWarning,


### Task 3: Monitor your data with automatic data profiling
Enable automated data profiling in your pipeline.

In [35]:
job = aiplatform.PipelineJob(
        display_name=f"pipeline-run-deploy-endpoint",
        template_path=COMPILED_PIPELINE_FILE,
        pipeline_root=PIPELINE_ROOT,
        parameter_values={
# Enable here the data profiling. Have a look at the pipeline definition to understand how to do it!
           "" : ""
        },
    )
job.submit()





'''
Solution of pipeline triggered with data profiling enabled

job = aiplatform.PipelineJob(
        display_name=f"pipeline-run-deploy-endpoint",
        template_path=COMPILED_PIPELINE_FILE,
        pipeline_root=PIPELINE_ROOT,
        parameter_values={
           "profile_data" : "True"
        },
    )
job.submit()
'''

Creating PipelineJob


INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob


PipelineJob created. Resource name: projects/370018035372/locations/us-central1/pipelineJobs/custom-training-pipeline-20230123183954


INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/370018035372/locations/us-central1/pipelineJobs/custom-training-pipeline-20230123183954


To use this PipelineJob in another session:


...

pipeline_job = aiplatform.PipelineJob.get('projects/370018035372/locations/us-central1/pipelineJobs/custom-training-pipeline-20230123183954')


INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/370018035372/locations/us-central1/pipelineJobs/custom-training-pipeline-20230123183954')


View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/custom-training-pipeline-20230123183954?project=370018035372


INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/custom-training-pipeline-20230123183954?project=370018035372


'\nSolution of pipeline triggered with data profiling enabled\n\njob = aiplatform.PipelineJob(\n        display_name=f"pipeline-run-deploy-endpoint",\n        template_path=COMPILED_PIPELINE_FILE,\n        pipeline_root=PIPELINE_ROOT,\n        parameter_values={\n           "profile_data" : "True"\n        },\n    )\njob.submit()\n'

### Task 4: Prepare for production 
Once you are ready to schedule a certain pipeline to run in production, you can enable the pipeline to push the created model to the Model Registry and then deploy the model to an Endpoint if the MAE is below 100. 

In [ ]:
job = aiplatform.PipelineJob(
        display_name=f"{EXPERIMENT_NAME}-pipeline-run-{i}",
        template_path=COMPILED_PIPELINE_FILE,
        pipeline_root=PIPELINE_ROOT,
        parameter_values={
# Customise the pipeline parameters to enable data_profiling and pushing a model to the endpoint.
            "":"",
            "":""
        },
    )
# We trigger the pipeline synchrounosly so that we run the next cell only when this pipeline is completed.
job.run()



'''
Solution of pipeline triggered with data profiling enabled

job = aiplatform.PipelineJob(
        display_name=f"{EXPERIMENT_NAME}-pipeline-run-{i}",
        template_path=COMPILED_PIPELINE_FILE,
        pipeline_root=PIPELINE_ROOT,
        parameter_values={
            "profile_data":"True",
            "push_model":"True"    
        },
    )
job.run()
'''

## Send request to the Endpoint

Once the above pipeline is completed, you will be able to test your online serving endpoint. 
You can do that with the following code:

In [37]:
from typing import Union, List, Dict
from datetime import datetime, timedelta
import time
import random

from google.cloud import aiplatform as vertex_ai
from google.cloud.aiplatform import model_monitoring

ENDPOINT_NAME= "mlops_model_endpoint"

endpoint = vertex_ai.Endpoint.list(filter=f"display_name={ENDPOINT_NAME}")[-1]

In [38]:
import random


def get_request():
#     We generate a list of 9 random numbers, feel free to customise the input here!
    request =  [random.sample(range(1,10),9)]
    return request

for online_sample in range(1,10):
    request = get_request()
    print(f"Request input {request}")
    prediction = endpoint.predict(request)

    print(f"Predicted value {prediction.predictions}")


Request input [[5, 7, 8, 3, 4, 6, 9, 1, 2]]
Predicted value [[15.9476852]]
Request input [[3, 2, 7, 5, 1, 9, 4, 8, 6]]
Predicted value [[21.0523109]]
Request input [[2, 5, 7, 9, 1, 3, 6, 8, 4]]
Predicted value [[15.3422489]]
Request input [[3, 9, 2, 5, 8, 4, 7, 6, 1]]
Predicted value [[14.421196]]
Request input [[4, 5, 3, 7, 6, 8, 2, 1, 9]]
Predicted value [[22.3368282]]
Request input [[4, 6, 3, 9, 2, 8, 7, 1, 5]]
Predicted value [[20.2386265]]
Request input [[8, 2, 4, 6, 3, 1, 7, 9, 5]]
Predicted value [[19.5185356]]
Request input [[4, 3, 5, 9, 6, 8, 7, 1, 2]]
Predicted value [[21.3011894]]
Request input [[8, 2, 1, 5, 4, 7, 3, 6, 9]]
Predicted value [[23.8133774]]


You can do the same test in the Console: 
1. Visit the [Endpoint page](https://console.cloud.google.com/vertex-ai/endpoints)
2. Click on the model endpoint `mlops_model_endpoint`
3. Click on the model you want to test, in this case `mlops_model`
4. Go to `Deploy & Test`
5. Test by sending a request. You can copy past for example the following one `{"instances": [[1,2,3,4,5,6,7,8,9]]}`
